In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [6]:
df = pd.read_csv('train (1).csv')

In [ ]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [19]:
df.drop(columns=['Name'],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [21]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [23]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [25]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [27]:
numerical_features = ['Age','Fare']
numerical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])
categorical_features = ['Embarked','Sex']
categorical_transformer = Pipeline([
    ('imputer',SimpleImputer(strategy='most-frequent')),
    ('ohe',OneHotEncoder(handle_unknown='ignore'))
])

In [29]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_features),
        ('cat',categorical_transformer,categorical_features)
    ]
)

In [31]:
from sklearn import set_config

In [35]:
clf = Pipeline(
    steps=[
        ('preprocessor',preprocessor),
        ('classifier',LogisticRegression())
    ]
)

In [37]:
set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most-frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('classifier', LogisticRegression())])

In [39]:
param_grid = {
    'preprocessor__num__imputer__strategy' : ['mean','median'],
    'preprocessor__cat__imputer__strategy' : ['most_frequent','constant'],
    'classifier__C' : [0.1,1.0,10,100]
}

grid_search = GridSearchCV(clf,param_grid,cv=10)

In [41]:
grid_search.fit(x_train,y_train)
print(f"Best params: ")
print(grid_search.best_params_)

Best params: 
{'classifier__C': 0.1, 'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'mean'}


In [43]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.784


In [49]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values('mean_test_score',ascending=False)
cv_results[['param_classifier__C','param_preprocessor__cat__imputer__strategy','param_preprocessor__num__imputer__strategy','mean_test_score']]

,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,mean_test_score
0,0.1,most_frequent,mean,0.783725
1,0.1,most_frequent,median,0.783725
2,0.1,constant,mean,0.783725
3,0.1,constant,median,0.783725
4,1.0,most_frequent,mean,0.782316
5,1.0,most_frequent,median,0.782316
6,1.0,constant,mean,0.782316
7,1.0,constant,median,0.782316
8,10,most_frequent,mean,0.782316
9,10,most_frequent,median,0.782316
